In [1]:
%matplotlib inline

import collections
import pprint

import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

# Summary of Problem

Ten Perfectly Rational Pirate Logicians (PRPLs) find 10 (indivisible) gold pieces and wish to distribute the booty among themselves.

The pirates each have a unique rank, from the captain on down. The captain puts forth the first plan to divide up the gold, whereupon the pirates (including the captain) vote. If at least half the pirates vote for the plan, it is enacted, and the gold is distributed accordingly. If the plan gets fewer than half the votes, however, the captain is killed, the second-in-command is promoted, and the process starts over. (They’re mutinous, these PRPLs.)

Pirates always vote by the following rules, with the earliest rule taking precedence in a conflict:

1. Self-preservation: A pirate values his life above all else.
1. Greed: A pirate seeks as much gold as possible.
1. Bloodthirst: Failing a threat to his life or bounty, a pirate always votes to kill.

Under this system, how do the PRPLs divide up their gold?

Extra credit: Solve the generalized problem where there are P pirates and G gold pieces.



## preliminary thoughts

This strikes me as an induction algorithm. Assume we know how $n_p$ pirates will divvy up $n_c$ gold coins. Call this distribution of coins $d^{n_p}$, and in it each pirate gets $d_i^{n_p}$. Can we determine from that distribution how $n_p + 1$ pirates will divvy up their gold?

Given the previous best value, the above voting logic can be broken down as follows:

$$
\textrm{vote} = \left\{
\begin{array}{rl}
    \textrm{AYE AYE} & i = n_p + 1 \textrm{ (because of self-preservation)} \\
    \textrm{AYE AYE} & d_i^{n_p + 1} > d_i^{n_p} \textrm{ (because of greed)} \\
    \textrm{ARGH} & d_i^{n_p + 1} \leq d_i^{n_p} \textrm{ (because of greed (less than) and bloodthirst (equal))} \\
\end{array}
\right.
$$

The base cases of $n=1$ and $n=2$ are easily solved (in both, the captain pirate takes all $n_c$ coins), so this should be fairly easy to iterate.

In [2]:
AYE_AYE = True
MURRRRRRRDER = False

class Pirate(object):
    num = 0
    @classmethod
    def reset_numbers(cls):
        Pirate.num = 0
        
    def __init__(self, isCaptain=False):
        self.number = Pirate.num
        Pirate.num += 1
        self.isCaptain = isCaptain
        self.previous_best = collections.defaultdict(int)
        
    def me_vote_is(self, distribution):
        return self.isCaptain or (distribution[self.number] > self.previous_best[len(distribution) - 1])
    
    def ye_get_what_i_give_ye(self, numCoins, numPirates):
        self.previous_best[numPirates] = numCoins

In [3]:
class Crew(object):
    def __init__(self, num):
        Pirate.reset_numbers()
        self.optimal = {}
        self.pirates = [Pirate() for i in range(num)]
        
    def new_captain(self):
        for p in self.pirates:
            p.isCaptain = False
        c = Pirate(isCaptain=True)
        self.captain = c
        self.pirates.append(self.captain)
        
    def find_optimal_distribution(self, numCoins):
        self.optimal[self.num_pirates] = {
            'capnsGold': 0,
            'dist': None
        }
        for dist in self.distributions(numCoins):
            votes = [p.me_vote_is(dist) for p in self.pirates]
            hasVotes = sum(votes) >= (self.num_pirates / 2)
            capnsHappy = dist[-1] > self.optimal[self.num_pirates]['capnsGold']
            if hasVotes and capnsHappy:
                self.optimal[self.num_pirates] = {
                    'capnsGold': dist[-1],
                    'dist': dist
                }
        # update based on best distribution possible
        for (p, d) in zip(self.pirates, self.optimal[self.num_pirates]['dist']):
            p.ye_get_what_i_give_ye(d, self.num_pirates)
    
    @property
    def num_pirates(self):
        return len(self.pirates)
    
    def distributions(self, numCoins, numPirates=None):
        if numPirates is None:
            numPirates = self.num_pirates
        # base cases:
        if numPirates == 0:
            return
        elif numPirates == 1:
            yield [numCoins]
        else:
            for capnNum in range(numCoins + 1):
                leftOver = numCoins - capnNum
                for subDist in self.distributions(leftOver, numPirates - 1):
                    yield [capnNum] + subDist
    
    def iterate(self, numCoins, verbose=False):
        self.new_captain()
        self.find_optimal_distribution(numCoins)
        if verbose:
            pprint.pprint(self.optimal)

        
def solve(numPirates, numCoins):
    c = Crew(1)
    c.find_optimal_distribution(numCoins=numCoins)
    for i in range(1, numPirates):
        c.iterate(numCoins, verbose=(i == (numPirates - 1)))
    return c

In [4]:
solve(numPirates=10, numCoins=10)

{1: {'capnsGold': 10, 'dist': [10]},
 2: {'capnsGold': 10, 'dist': [0, 10]},
 3: {'capnsGold': 9, 'dist': [1, 0, 9]},
 4: {'capnsGold': 9, 'dist': [0, 1, 0, 9]},
 5: {'capnsGold': 8, 'dist': [1, 0, 1, 0, 8]},
 6: {'capnsGold': 8, 'dist': [0, 1, 0, 1, 0, 8]},
 7: {'capnsGold': 7, 'dist': [1, 0, 1, 0, 1, 0, 7]},
 8: {'capnsGold': 7, 'dist': [0, 1, 0, 1, 0, 1, 0, 7]},
 9: {'capnsGold': 6, 'dist': [1, 0, 1, 0, 1, 0, 1, 0, 6]},
 10: {'capnsGold': 6, 'dist': [0, 1, 0, 1, 0, 1, 0, 1, 0, 6]}}
